In [421]:
from random import random, choice, randint
import operator
from functools import reduce #for use reduce function
from collections import namedtuple
from queue import PriorityQueue #for the frontier
import numpy as np
from copy import copy

In [422]:
PROBLEM_SIZE = 10
NUM_SETS = 30

#this line create an array of "PROBLEM_SIZE" size with random boolean value
#np.array([random() < .2 for _ in range(PROBLEM_SIZE)])
#This line create a set that have the total number of sets and put it in a tuple
SETS = tuple ( ( np.array([random() < .2 for _ in range(PROBLEM_SIZE)]) for _ in range(NUM_SETS)) )
#for know if a state is taken or not we use a tuple
State = namedtuple('State', ['taken', 'not_taken'])

In [423]:
'''
#For better undestand the goal_check metod

#i take all the sets in the position specified in the state[0] (1, 3, 5)
[sets[i] for i in state[0]]

#sum all the element taken
#sum([sets[i] for i in state[0]])

#same as the previuos line but with reduce that consent to obtain result a vector of true/false values (not in integer like the previous one)
#reduce(np.logical_or, [sets[i] for i in state[0]])

#get only a true o false (true if all is true or false if one is false)
np.all(reduce(np.logical_or, [sets[i] for i in state[0]] ))
'''

'\n#For better undestand the goal_check metod\n\n#i take all the sets in the position specified in the state[0] (1, 3, 5)\n[sets[i] for i in state[0]]\n\n#sum all the element taken\n#sum([sets[i] for i in state[0]])\n\n#same as the previuos line but with reduce that consent to obtain result a vector of true/false values (not in integer like the previous one)\n#reduce(np.logical_or, [sets[i] for i in state[0]])\n\n#get only a true o false (true if all is true or false if one is false)\nnp.all(reduce(np.logical_or, [sets[i] for i in state[0]] ))\n'

In [424]:
def evaluate(state):
  cost = sum(state)
  #return np.all(reduce(np.logical_or, [SETS[i] for i in state[0]] ))
  #thanks to namedtuple we can call state[0] element state.taken
  #return np.all(reduce(np.logical_or, [SETS[i] for i in state.taken] ))
  #we need to work with also 0 in initial state.taken elements so:
  valid = np.all(
    reduce(
      np.logical_or, [SETS[i] for i, t in enumerate(state) if t], np.array([False for _ in range(PROBLEM_SIZE)]) 
    )
  )
  return valid, -cost if valid else 0

In [425]:
def tweak(state):
    new_state = copy(state)
    index = randint(0, PROBLEM_SIZE-1)
    new_state[index] = not new_state[index]
    return new_state

In [426]:
current_state = [choice([True, False]) for _ in range(PROBLEM_SIZE)]
print(current_state, evaluate(current_state))

for step in range(100):
    new_state = tweak(current_state)
    if evaluate(new_state) > evaluate(current_state):
        current_state = new_state
        print(current_state, evaluate(current_state))
    

[True, True, False, False, False, True, False, False, False, True] (False, 0)


In [427]:
#to prevently check if the problem have a solution
state_all = State(set(range(NUM_SETS)), set())
assert goal_check( state_all ), "Problem not solvable"

In [428]:
#we can use different que type
frontier = PriorityQueue()
#start state
start_state = State(set(), set(range(NUM_SETS)))

#put the start state in the frontier
frontier.put(start_state)

In [429]:
#get the first element for the fronier
current_state = frontier.get()

#check if with the first state we can already have a solution
if goal_check(current_state):
  print("WOW solution find")

counter = 0
while not goal_check(current_state):
  counter += 1
  for action in current_state[1]:
    #we need to create a new state taking an element
    #new_state = State(current_state.taken | {action}, current_state.not_taken - {action})
    #same as the previous but with ^
    new_state = State(current_state.taken ^ {action}, current_state.not_taken ^ {action})
    #i need to put the new state in the frontier
    frontier.put(new_state)
  current_state = frontier.get()

print(f"solved in {counter:,} steps")

solved in 77 steps


In [430]:
current_state = [False for _ in range(PROBLEM_SIZE)]